In this notebook, we'll examine the UCI electricity demand dataset (see [here][0]), and set up a dataframe that can be used for time series prediction testing. 

[0]:https://archive.ics.uci.edu/dataset/321/electricityloaddiagrams20112014

Note that this notebook is for documentation only, the model data can be updated by running `zero-time-series-demo prep ----model-data-pipeline`

In [1]:
%load_ext autoreload
%autoreload 2

# Load Data

In [2]:
from zero_ts_demo.imports import *
from zero_ts_demo import helpers as hp
from zero_ts_demo.load_data import load_model_data

In [3]:
MODEL_DATA = load_model_data()
df = MODEL_DATA.copy()
df.head()

,mt_004,mt_005,mt_006,mt_007,mt_009,mt_010,mt_011,mt_013,mt_014,mt_015,...,mt_360,mt_361,mt_362,mt_363,mt_364,mt_365,mt_366,mt_367,mt_368,mt_370
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Prophet

In [5]:
from prophet import Prophet

In [28]:
col = pd.Series(df.columns).sample().iloc[0]
perc_test = 0.1

In [48]:
# create dataframe for prophet
sl = df[[col]].reset_index().copy()
sl.columns = ['ds', 'y']
m = sl['y'] > 0

# todo - use full dataframe, dropping here for testing 
sl = sl[m].tail(1000).copy()

idx = 10
idx_train = sl.index[:-idx]
idx_test = sl.index[-idx:]
sl_train = sl.loc[idx_train]
sl_test = sl.loc[idx_test]

In [49]:
model = Prophet()
model.fit(sl_train)

04:55:58 - cmdstanpy - INFO - Chain [1] start processing
04:55:58 - cmdstanpy - INFO - Chain [1] done processing


In [58]:
model.predict(sl_test.drop(columns='y')).merge(sl_test, on=['ds'])

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y
0,2014-12-31 21:45:00,2449.120750,2180.256070,3127.422910,2449.120750,2449.120750,212.589645,212.589645,212.589645,212.589645,212.589645,212.589645,0.0,0.0,0.0,2661.710395,1874.492754
1,2014-12-31 22:00:00,2446.421129,2127.937159,2999.746372,2446.421129,2446.421129,109.547330,109.547330,109.547330,109.547330,109.547330,109.547330,0.0,0.0,0.0,2555.968459,1903.478261
2,2014-12-31 22:15:00,2443.721507,1980.810763,2872.654514,2443.721507,2443.721507,4.703143,4.703143,4.703143,4.703143,4.703143,4.703143,0.0,0.0,0.0,2448.424650,1917.971014
3,2014-12-31 22:30:00,2441.021886,1846.518199,2789.074654,2441.021886,2441.021886,-101.185486,-101.185486,-101.185486,-101.185486,-101.185486,-101.185486,0.0,0.0,0.0,2339.836400,1787.536232
4,2014-12-31 22:45:00,2438.322264,1786.292063,2662.413563,2438.322264,2438.322264,-207.503002,-207.503002,-207.503002,-207.503002,-207.503002,-207.503002,0.0,0.0,0.0,2230.819263,1932.463768
5,2014-12-31 23:00:00,2435.622643,1681.207213,2576.317540,2435.622643,2435.622643,-313.749039,-313.749039,-313.749039,-313.749039,-313.749039,-313.749039,0.0,0.0,0.0,2121.873603,1874.492754
6,2014-12-31 23:15:00,2432.923021,1569.843808,2504.886905,2432.923021,2432.923021,-419.501507,-419.501507,-419.501507,-419.501507,-419.501507,-419.501507,0.0,0.0,0.0,2013.421515,1787.536232
7,2014-12-31 23:30:00,2430.223400,1408.370417,2346.465769,2430.163102,2430.223400,-524.372969,-524.372969,-524.372969,-524.372969,-524.372969,-524.372969,0.0,0.0,0.0,1905.850431,1613.478261
8,2014-12-31 23:45:00,2427.523778,1342.392507,2243.271627,2427.256127,2427.579695,-627.964342,-627.964342,-627.964342,-627.964342,-627.964342,-627.964342,0.0,0.0,0.0,1799.559436,1671.449275
9,2015-01-01 00:00:00,2424.824157,1240.568636,2146.641315,2424.340688,2425.067765,-729.820021,-729.820021,-729.820021,-729.820021,-729.820021,-729.820021,0.0,0.0,0.0,1695.004136,1482.898551
